In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_rel
import statsmodels.api as sm

reaction_df = pd.read_csv('/content/drive/MyDrive/reaction_time_datas - reaction_time_datas - Sheet1 (1) (2) - reaction_time_datas - reaction_time_datas - Sheet1 (1) (2).csv')
digit_df = pd.read_csv('/content/drive/MyDrive/processed_digit_span_data_last - processed_digit_span_data (1) - processed_digit_span_data_last - processed_digit_span_data (1).csv')
stroop_df = pd.read_csv('/content/drive/MyDrive/Stroop_score_datas - Sheet1 (3).csv')
verbal_df = pd.read_csv('/content/drive/MyDrive/verbal_test_data (2).csv')


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/reaction_time_datas - reaction_time_datas - Sheet1 (1) (2) - reaction_time_datas - reaction_time_datas - Sheet1 (1) (2).csv'

In [ ]:
# Prepare Reaction Time
reaction_df.columns = reaction_df.columns.str.strip()
reaction_df['Date'] = pd.to_datetime(reaction_df['Date'], dayfirst=True)
reaction_df['Reaction Time (Before)'] = reaction_df['Reaction Time (Before)'].str.replace(',', '.').astype(float)
reaction_df['Reaction Time (After)'] = reaction_df['Reaction Time (After)'].str.replace(',', '.').astype(float)
reaction_df['Reaction Improvement'] = reaction_df['Reaction Time (Before)'] - reaction_df['Reaction Time (After)']

In [ ]:
# Prepare Digit Span
digit_df['Date'] = pd.to_datetime(digit_df['Date'])
digit_df['Digit Span Improvement'] = digit_df['Digit Span (After)'] - digit_df['Digit Span (Before)']

In [ ]:
# Prepare Stroop
stroop_df_clean = stroop_df.iloc[2:].copy()
stroop_df_clean.columns = ['Date', 'Chess Played?', 'Stroop Score (Before)', 'Stroop Score (After)']
stroop_df_clean['Date'] = pd.to_datetime(stroop_df_clean['Date'], dayfirst=True)
stroop_df_clean['Stroop Score (Before)'] = stroop_df_clean['Stroop Score (Before)'].astype(float)
stroop_df_clean['Stroop Score (After)'] = stroop_df_clean['Stroop Score (After)'].astype(float)
stroop_df_clean['Stroop Improvement'] = stroop_df_clean['Stroop Score (After)'] - stroop_df_clean['Stroop Score (Before)']

In [ ]:
# Prepare Verbal Memory
verbal_df['Date'] = pd.to_datetime(verbal_df['Date'], dayfirst=True)
verbal_pivot = verbal_df.pivot(index=['Date', 'Chess Played?'], columns='Test Phase', values='Verbal Memory Score').reset_index()
verbal_pivot['Verbal Improvement'] = verbal_pivot['After'] - verbal_pivot['Before']

In [ ]:
# Reaction Time
reaction_t, reaction_p = ttest_rel(reaction_df['Reaction Time (Before)'], reaction_df['Reaction Time (After)'], nan_policy='omit')
# Digit Span
digit_t, digit_p = ttest_rel(digit_df['Digit Span (Before)'], digit_df['Digit Span (After)'], nan_policy='omit')
# Stroop
stroop_t, stroop_p = ttest_rel(stroop_df_clean['Stroop Score (Before)'], stroop_df_clean['Stroop Score (After)'], nan_policy='omit')
# Verbal Memory
verbal_t, verbal_p = ttest_rel(verbal_pivot['Before'], verbal_pivot['After'], nan_policy='omit')

print(f'Reaction Time Paired t-test p-value: {reaction_p:.4f}')
print(f'Digit Span Paired t-test p-value: {digit_p:.4f}')
print(f'Stroop Test Paired t-test p-value: {stroop_p:.4f}')
print(f'Verbal Memory Paired t-test p-value: {verbal_p:.4f}')

In [ ]:
# Add Chess Played as binary
reaction_df['Chess_Yes'] = reaction_df['Chess Played?'].apply(lambda x: 1 if x == 'Yes' else 0)
digit_df['Chess_Yes'] = digit_df['Chess Played?'].apply(lambda x: 1 if x == 'Yes' else 0)
stroop_df_clean['Chess_Yes'] = stroop_df_clean['Chess Played?'].apply(lambda x: 1 if x == 'Yes' else 0)
verbal_pivot['Chess_Yes'] = verbal_pivot['Chess Played?'].apply(lambda x: 1 if x == 'Yes' else 0)

In [ ]:
import statsmodels.api as sm

# Reaction Time
X_reaction = sm.add_constant(reaction_df['Chess_Yes'])
model_reaction = sm.OLS(reaction_df['Reaction Improvement'], X_reaction).fit()

# Digit Span
X_digit = sm.add_constant(digit_df['Chess_Yes'])
model_digit = sm.OLS(digit_df['Digit Span Improvement'], X_digit).fit()

# Stroop
X_stroop = sm.add_constant(stroop_df_clean['Chess_Yes'])
model_stroop = sm.OLS(stroop_df_clean['Stroop Improvement'], X_stroop).fit()

# Verbal Memory
X_verbal = sm.add_constant(verbal_pivot['Chess_Yes'])
model_verbal = sm.OLS(verbal_pivot['Verbal Improvement'], X_verbal).fit()

# Temiz ve basit çıktı almak
def simple_regression_output(model, model_name):
    try:
        p_value = model.pvalues['Chess_Yes']
        coef = model.params['Chess_Yes']

        print(f"🔹 {model_name}")
        if pd.isna(p_value) or pd.isna(coef):
            print(f"    🚫 Result: Regression could not be performed (possible missing or constant data).\n")
        else:
            print(f"    ➔ Coefficient (Chess Played): {coef:.4f}")
            print(f"    ➔ p-value: {p_value:.4f}")
            if p_value < 0.05:
                print(f"     Result: Statistically significant at 0.05 level.\n")
            else:
                print(f"     Result: Not statistically significant at 0.05 level.\n")
    except Exception as e:
        print(f"     Error analyzing {model_name}: {str(e)}\n")

# Kullanım:
simple_regression_output(model_reaction, "Reaction Time Improvement")
simple_regression_output(model_digit, "Digit Span Improvement")
simple_regression_output(model_stroop, "Stroop Improvement")
simple_regression_output(model_verbal, "Verbal Memory Improvement")
